### Hello World using Flask


**Key Changes from Python 2**

- print within parentheses e.g.print` 'hello'` to print`('hello')`
- added `mode='rb'` while operning the binary file. e.g open(scaler_filepath) to open(scaler_filepath, `mode='rb'`) 
- replaced `.as_matrix()` with `.values` e.g. df[columns]`.as_matrix()`.astype('float') to df[columns]`.values`.astype('float') 

**Other Issues you may encounter**
- Sometime running the API script with `debug=True` can lead to `No file found` error. Try setting `debug=False` in those cases.

In [1]:
## install packages if not installed already : You need to restart the kernel if new library is installed 
!pip install --user numpy pandas matplotlib seaborn scikit-learn flask version_information

In [ ]:
# quick function to restart the terminal. Alternative you can use menu : kernel -> restart
from IPython.display import display_html
def restartkernel() :
    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)
    
restartkernel()

In [1]:
import os
hello_world_script_file = os.path.join(os.path.pardir,'src','models','hello_world_api.py')

In [2]:
%%writefile $hello_world_script_file

from flask import Flask, request

app = Flask(__name__)

@app.route('/api', methods=['POST'])
def say_hello():
    data = request.get_json(force=True)
    name = data['name']
    return "hello {0}".format(name)

if __name__ == '__main__':
    app.run(port=10001, debug=False)

Writing ../src/models/hello_world_api.py


#### Running the Flask Server

**Option 1** -  if you are running from your local machine. Navigate to the "titanic/src/models" directory and use
`python3 hello_world_api.py` to run the flask server.Once it is running you can execute the next steps. 

**Option 2** - if you are using sandbox environment or want to you jupyter notebook environment then you can use the terminal feature available in jupyter notebook.   

- Go to jupyter home page ( click on jupyter logo on the top ) in a separte tab. 
- Click on `New` drop down and select `Terminal`. It will open the terminal or shell enviornment.
- Navigate to current module directory and then to "titanic/src/models" using `cd` command. Use `ls` command to validate if you have `hello_world_api.py` file in the current directory. 
- Run `python3 hello_world_api.py` to run the flask server. once the server is running in a separate tab, you can execute the next step in this notebook. 

Note : if the port mentioned in the flask code `10001` is not available or being used, try another port number.

In [3]:
import json
import requests

In [4]:
url = 'http://127.0.0.1:10001/api'
data = json.dumps({'name' : 'abhi'})
r = requests.post(url,data)

In [5]:
print(r.text)

hello abhi


### Machine Learning API using Flask

#### Building API

In [6]:
machine_learning_api_script_file = os.path.join(os.path.pardir,'src','models','machine_learning_api.py')

In [7]:
%%writefile $machine_learning_api_script_file

from flask import Flask, request
import pandas as pd
import numpy as np
import json
import pickle
import os

app = Flask(__name__)

# Load Model and Scaler Files
model_path = os.path.join(os.path.pardir,os.path.pardir,'models')
model_filepath = os.path.join(model_path, 'lr_model.pkl')
scaler_filepath = os.path.join(model_path, 'lr_scaler.pkl')

scaler = pickle.load(open(scaler_filepath, mode='rb')) # CHANGE : ADDED mode='rb'
model = pickle.load(open(model_filepath, mode='rb')) # CHANGE : ADDED mode='rb'

# columns
columns = [ u'Age', u'Fare', u'FamilySize', \
       u'IsMother', u'IsMale', u'Deck_A', u'Deck_B', u'Deck_C', u'Deck_D', \
       u'Deck_E', u'Deck_F', u'Deck_G', u'Deck_Z', u'Pclass_1', u'Pclass_2', \
       u'Pclass_3', u'Title_Lady', u'Title_Master', u'Title_Miss', u'Title_Mr', \
       u'Title_Mrs', u'Title_Officer', u'Title_Sir', u'Fare_Bin_very_low', \
       u'Fare_Bin_low', u'Fare_Bin_high', u'Fare_Bin_very_high', u'Embarked_C', \
       u'Embarked_Q', u'Embarked_S', u'AgeState_Adult', u'AgeState_Child'] 


@app.route('/api', methods=['POST'])
def make_prediction():
    # read json object and conver to json string
    data = json.dumps(request.get_json(force=True))
    # create pandas dataframe using json string
    df = pd.read_json(data)
    # extract passengerIds
    passenger_ids = df['PassengerId'].ravel()
    # actual survived values
    actuals = df['Survived'].ravel()
    # extract required columns based and convert to matrix
    X = df[columns].values.astype('float') # CHANGE to .values
    # transform the input
    X_scaled = scaler.transform(X)
    # make predictions
    predictions = model.predict(X_scaled)
    # create response dataframe
    df_response = pd.DataFrame({'PassengerId': passenger_ids, 'Predicted' : predictions, 'Actual' : actuals})
    # return json 
    return df_response.to_json()
 

if __name__ == '__main__':
    # host flask app at port 10001
    app.run(port=10001, debug=False)

Writing ../src/models/machine_learning_api.py


#### Running the Flask Server

**Option 1** -  if you are running from your local machine. Navigate to the "titanic/src/models" directory and use
`python3 machine_learning_api.py` to run the flask server.Once it is running you can execute the next steps. 

**Option 2** - if you are using sandbox environment or want to you jupyter notebook environment then you can use the terminal feature available in jupyter notebook. 

if you are already running the flask server from the previous step. You can simply stop the server using `CTRL+C` and run `python3 machine_learning_api.py` to run the flask server. once the server is running in a separate tab, you can execute the next step in this notebook. 

otherwise

- Go to jupyter home page ( click on jupyter logo on the top ) 
- Click on `New` drop down and select `Terminal`. It will open the terminal or shell enviornment.
- Navigate to current module directory and then to "titanic/src/models" using `cd` command. Use `ls` command to validate if you have `machine_learning_api.py` file in the current directory. 
- Run `python3 machine_learning_api.py` to run the flask server. once the server is running in a separate tab, you can execute the next step in this notebook. 

Note : if the port mentioned in the flask code `10001` is not available or being used, try another port number.

#### Invoking API using Requests

In [8]:
import os
import pandas as pd
processed_data_path = os.path.join(os.path.pardir,'data','processed')
train_file_path = os.path.join(processed_data_path, 'train.csv')
train_df = pd.read_csv(train_file_path)

In [9]:
survived_passengers = train_df[train_df['Survived'] == 1][:5]

In [10]:
survived_passengers

,PassengerId,Survived,Age,Fare,FamilySize,IsMother,IsMale,Deck_A,Deck_B,Deck_C,...,Title_Sir,Fare_Bin_very_low,Fare_Bin_low,Fare_Bin_high,Fare_Bin_very_high,Embarked_C,Embarked_Q,Embarked_S,AgeState_Adult,AgeState_Child
1,2,1,38.0,71.2833,2,0,0,0,0,1,...,0,0,0,0,1,1,0,0,1,0
2,3,1,26.0,7.9250,1,0,0,0,0,0,...,0,0,1,0,0,0,0,1,1,0
3,4,1,35.0,53.1000,2,0,0,0,0,1,...,0,0,0,0,1,0,0,1,1,0
8,9,1,27.0,11.1333,3,1,0,0,0,0,...,0,0,1,0,0,0,0,1,1,0
9,10,1,14.0,30.0708,2,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,1


In [11]:
import requests
def make_api_request(data):
    # url for api
    url = 'http://127.0.0.1:10001/api'
    # make post request
    r = requests.post(url,data)
    #return r.text
    return r.json()
    

In [12]:
make_api_request(survived_passengers.to_json())

{'PassengerId': {'0': 2, '1': 3, '2': 4, '3': 9, '4': 10},
 'Predicted': {'0': 1, '1': 1, '2': 1, '3': 1, '4': 1},
 'Actual': {'0': 1, '1': 1, '2': 1, '3': 1, '4': 1}}

In [13]:
# compare results
result = make_api_request(train_df.to_json())
df_result = pd.read_json(json.dumps(result))
df_result.head()

,PassengerId,Predicted,Actual
0,1,0,0
1,2,1,1
2,3,1,1
3,4,1,1
4,5,0,0


In [14]:
# accuracy level
import numpy as np
np.mean(df_result.Actual == df_result.Predicted)

0.8395061728395061